# Dropout

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

def dropout(X, drop_prob):
    X = X.float()
    assert 0 <= drop_prob <= 1  #结果不在这里面的话就产生断言
    keep_prob = 1 - drop_prob   #保留的概率=1-drop的概率
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:  
        return torch.zeros_like(X)  #如果保留概率为0，直接返回输入大小的全0
    mask = (torch.rand(X.shape) < keep_prob).float()    #

    return mask * X / keep_prob


In [3]:
X=torch.arange(16).view(2,8)
dropout(X,0)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [4]:
dropout(X,0.5)

tensor([[ 0.,  2.,  0.,  0.,  8., 10., 12., 14.],
        [ 0., 18., 20., 22., 24., 26.,  0.,  0.]])